In [126]:
import numpy as np
import pandas as pd
import os, itertools, csv
from bs4 import BeautifulSoup
import re
from pytrends.request import TrendReq
from datetime import *
import time

# load the dataset
dataset = pd.read_csv('./data/all_data.csv')
dataset.head()
dataset.shape

(39490, 31)

In [127]:
dataset = dataset.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
dataset.head()

,Id,Popularity,article_polarity,article_subjectivity,n_herf,n_image,n_self_herf,n_title_token,n_topic,timedelta,...,n_video,n_sentence,title_sentiment,LDA_0,LDA_1,LDA_2,LDA_3,LDA_4,n_article_token,unique_article_token
0,0,-1,0.126224,0.434848,14,0,0,9,7,2323,...,0,5,0.324056,0.203237,0.000879,0.000878,0.794140,0.000866,572,286
1,1,1,0.162588,0.511507,8,0,0,14,8,2406,...,0,8,0.239522,0.565165,0.001439,0.001433,0.019383,0.412581,299,152
2,2,1,0.030201,0.466191,4,0,0,12,5,2001,...,0,41,0.259172,0.000442,0.000445,0.998229,0.000441,0.000444,1059,539
3,3,-1,-0.033171,0.520430,7,0,0,5,4,2209,...,0,27,0.396490,0.001476,0.903604,0.091965,0.001469,0.001485,247,180
4,4,-1,0.081194,0.440452,9,50,0,10,5,2021,...,0,11,0.279841,0.000455,0.953835,0.044792,0.000460,0.000458,1311,664


In [128]:
dataset = dataset.drop(columns = ["Id", "weekday", "n_video"])
dataset.head()

,Popularity,article_polarity,article_subjectivity,n_herf,n_image,n_self_herf,n_title_token,n_topic,timedelta,title_polarity,...,channel_social_media,n_sentence,title_sentiment,LDA_0,LDA_1,LDA_2,LDA_3,LDA_4,n_article_token,unique_article_token
0,-1,0.126224,0.434848,14,0,0,9,7,2323,0.150000,...,0,5,0.324056,0.203237,0.000879,0.000878,0.794140,0.000866,572,286
1,1,0.162588,0.511507,8,0,0,14,8,2406,0.128788,...,0,8,0.239522,0.565165,0.001439,0.001433,0.019383,0.412581,299,152
2,1,0.030201,0.466191,4,0,0,12,5,2001,0.600000,...,0,41,0.259172,0.000442,0.000445,0.998229,0.000441,0.000444,1059,539
3,-1,-0.033171,0.520430,7,0,0,5,4,2209,-0.500000,...,0,27,0.396490,0.001476,0.903604,0.091965,0.001469,0.001485,247,180
4,-1,0.081194,0.440452,9,50,0,10,5,2021,0.300000,...,0,11,0.279841,0.000455,0.953835,0.044792,0.000460,0.000458,1311,664


In [46]:
dataset = dataset.drop(columns = ['author_Sam_Laird', 'author_Brian_Anthony_Hernandez', 
                                  'author_Neha_Prakash', 'author_Todd_Wasserman', 
                                  'author_Stan_Schroeder', 'author_Samantha_Murphy', 'author_Seth_Fiegerman'])
dataset.head()

,Popularity,article_polarity,article_subjectivity,n_herf,n_image,n_self_herf,n_title_token,n_topic,timedelta,title_polarity,...,n_video,n_article_token,n_sentence,title_sentiment,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
0,-1,0.126224,0.434848,14,0,0,9,7,2323,0.150000,...,0,583,5,0.324056,0.00,0.31,0.00,0.43,0.26,3
1,1,0.162588,0.511507,8,0,0,14,8,2406,0.128788,...,0,309,8,0.239522,0.34,0.08,0.00,0.57,0.00,3
2,1,0.030201,0.466191,4,0,0,12,5,2001,0.600000,...,0,1360,41,0.259172,0.06,0.00,0.08,0.00,0.86,4
3,-1,-0.033171,0.520430,7,0,0,5,4,2209,-0.500000,...,0,476,27,0.396490,0.00,0.27,0.62,0.00,0.10,2
4,-1,0.081194,0.440452,9,50,0,10,5,2021,0.300000,...,0,1945,11,0.279841,0.00,0.00,0.16,0.00,0.84,4


In [116]:
#dataset = dataset[["timedelta", "Popularity"]]

In [129]:
trainset = dataset.iloc[:27643, :]
testset = dataset.iloc[27643:, :]

In [118]:
trainset.shape

(27643, 2)

In [130]:
testset = testset.drop(columns=["Popularity"])
testset.head()

,article_polarity,article_subjectivity,n_herf,n_image,n_self_herf,n_title_token,n_topic,timedelta,title_polarity,title_subjectivity,...,channel_social_media,n_sentence,title_sentiment,LDA_0,LDA_1,LDA_2,LDA_3,LDA_4,n_article_token,unique_article_token
27643,-0.020425,0.441748,23,0,0,11,5,2241,0.0,0.0,...,0,22,0.259172,0.000880,0.257080,0.644346,0.096813,0.000881,516,319
27644,0.094010,0.408221,5,1,0,6,6,2189,0.0,0.0,...,0,7,0.396490,0.046589,0.003260,0.003256,0.003296,0.943600,135,81
27645,0.200833,0.512500,7,0,0,8,4,2317,0.0,0.0,...,0,6,0.680513,0.437150,0.002962,0.002949,0.002968,0.553970,157,106
27646,0.265590,0.462642,6,0,0,6,7,2449,0.0,0.0,...,0,10,0.371652,0.003009,0.420964,0.570072,0.002973,0.002982,153,113
27647,0.013742,0.334834,1,0,0,12,7,1852,0.0,0.0,...,0,8,0.538266,0.002273,0.333183,0.239286,0.422994,0.002265,215,155


In [131]:
X = trainset.drop(columns=['Popularity'])
y = trainset['Popularity']
y = y.astype(np.int)

In [132]:
from sklearn.model_selection import train_test_split
# X = np.array(X)
# y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train = np.array(X_train)
X_test = np.array(X_test)

In [10]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB

In [139]:
forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=90,
                                max_depth = 9,
                                random_state=1,
                                n_jobs=4)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_train)
print('Accuracy (forest): %.5f' % accuracy_score(y_train, y_pred))

y_pred = forest.predict(X_test)
print('Accuracy (forest): %.5f' % accuracy_score(y_test, y_pred))

# scores = cross_val_score(forest, X_train, y_train, cv=5)
# print('[%f-C]\nValidation accuracy: %.3f %s' % (10, scores.mean(), scores))

Accuracy (forest): 0.70672
Accuracy (forest): 0.54745


In [135]:
importances = forest.feature_importances_
# get sort indices in descending order
indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            X.columns.values[indices[f]], 
                            importances[indices[f]]))

 1) timedelta                      0.133541
 2) LDA_3                          0.068890
 3) LDA_4                          0.067661
 4) article_polarity               0.065006
 5) article_subjectivity           0.064766
 6) LDA_2                          0.060343
 7) LDA_0                          0.060210
 8) LDA_1                          0.057219
 9) n_article_token                0.054869
10) unique_article_token           0.052012
11) title_sentiment                0.051557
12) n_sentence                     0.040148
13) n_herf                         0.037704
14) title_polarity                 0.034548
15) n_topic                        0.030006
16) n_title_token                  0.028671
17) n_image                        0.028490
18) title_subjectivity             0.025607
19) channel_entertainment          0.007523
20) channel_social_media           0.007292
21) n_self_herf                    0.007012
22) channel_tech                   0.006290
23) channel_world               

In [50]:
X = np.array(X)
y = np.array(y)

In [134]:
tree = DecisionTreeClassifier(max_depth = 1, random_state = 0)

tree.fit(X_train, y_train)

y_pred = tree.predict(X_train)
print('Accuracy (forest): %.5f' % accuracy_score(y_train, y_pred))

y_pred = forest.predict(X_test)
print('Accuracy (forest): %.5f' % accuracy_score(y_test, y_pred))

Accuracy (forest): 0.53297
Accuracy (forest): 0.54805


# GOGOGOGOGOGO

In [24]:
y_pred = forest.predict(np.array(testset))

In [25]:
res = pd.DataFrame(y_pred)
#res.columns(['Popularity'])
ID = [i for i in range(27643, 27643 + res.shape[0])]
res['Id'] = ID
res.columns = ['Popularity', 'Id']
res = res.set_index('Id')
#res.columns(['Id', 'Popularity'])

In [26]:
res.head(20)

,Popularity
Id,
27643,-1
27644,-1
27645,-1
27646,1
27647,-1
27648,-1
27649,1
27650,1
27651,1


In [27]:
res.to_csv("./data/pred.csv")